# Donations Data

In [5]:
import pandas as pd
import requests
import time
from mezo.currency_utils import format_musd_currency_columns
from mezo.currency_config import MUSD_MARKET_MAP

In [38]:
# Contract addresses to fetch
contracts = {
    "Store": "0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE",
    "Donations": "0x6aD9E8e5236C0E2cF6D755Bb7BE4eABCbC03f76d"
}

# Store all transactions from both contracts
all_transactions = []

# Loop through both contracts
for contract_name, contract_address in contracts.items():
    print(f"\n{'='*60}")
    print(f"📡 Fetching transactions for {contract_name}")
    print(f"📍 Contract: {contract_address}")
    print(f"{'='*60}")
    
    url = f"https://api.explorer.mezo.org/api/v2/addresses/{contract_address}/transactions"
    transactions_list = []
    seen_hashes = set()
    next_page_params = None
    page_count = 0

    while True:
        page_count += 1
        print(f"Fetching page {page_count} for {contract_name}...")
        
        # Use cursor-based pagination
        if next_page_params:
            response = requests.get(url, params=next_page_params)
        else:
            response = requests.get(url)
        
        if response.status_code != 200:
            print(f"❌ Error: {response.status_code}")
            break
        
        data = response.json()
        transactions = data.get("items", [])
        
        if not transactions:
            print(f"No more transactions found for {contract_name}")
            break
        
        print(f"Found {len(transactions)} transactions on page {page_count}")
        
        # Process each transaction
        new_count = 0
        for tx in transactions:
            tx_hash = tx.get('hash')
            
            # Skip duplicates (safety check)
            if tx_hash in seen_hashes:
                continue
            
            seen_hashes.add(tx_hash)
            new_count += 1
            
            tx_data = {
                'contract_name': contract_name,
                'contract_address': contract_address,
                'timestamp_': tx.get('timestamp'),
                'method': tx.get('method'),
                'fee_value': int(tx.get('fee', {}).get('value', 0)) if tx.get('fee') else 0,
                'has_error': tx.get('has_error_in_internal_txs', False),
                'from_address': tx.get('from', {}).get('hash') if tx.get('from') else None,
                'to_address': tx.get('to', {}).get('hash') if tx.get('to') else None,
                'transactionHash_': tx_hash,
                'block_number': tx.get('block'),
            }

            # Extract decoded input parameters if available
            if tx.get('decoded_input') and tx.get('decoded_input').get('parameters'):
                parameters = tx.get('decoded_input').get('parameters')
                if len(parameters) > 0:
                    first_param = parameters[0]
                    tx_data['param_0_name'] = first_param.get('name')
                    tx_data['param_0_value'] = str(first_param.get('value'))
            
            all_transactions.append(tx_data)
        
        print(f"✅ Added {new_count} new transactions for {contract_name}")
        
        # Get next page cursor
        next_page_params = data.get("next_page_params")
        
        if not next_page_params:
            print(f"📄 Reached last page for {contract_name}")
            break
        
        # Be respectful to the API
        time.sleep(0.1)
    
    # Contract summary
    print(f"\n📊 {contract_name} Summary:")
    print(f"  Total pages: {page_count}")
    print(f"  Total transactions: {len(transactions_list)}")
    
    # Add delay between contracts
    if contract_name != list(contracts.keys())[-1]:
        print(f"\n⏳ Waiting before fetching next contract...")
        time.sleep(1)

# Create combined DataFrame
print(f"\n{'='*60}")
print(f"📊 COMBINED ANALYSIS")
print(f"{'='*60}")

if all_transactions:
    transactions_df = pd.DataFrame(all_transactions)
    
    # Sort by timestamp (most recent first)
    transactions_df['timestamp_dt'] = pd.to_datetime(transactions_df['timestamp_'])
    transactions_df = transactions_df.sort_values('timestamp_dt', ascending=False)
    
    # Overall statistics
    total_transactions = len(transactions_df)
    contracts_analyzed = transactions_df['contract_name'].nunique()
    date_range = f"{transactions_df['timestamp_'].min()} to {transactions_df['timestamp_'].max()}"
    total_fees = transactions_df['fee_value'].sum()
    unique_methods = transactions_df['method'].nunique()
    
    print(f"✅ Total transactions collected: {total_transactions}")
    print(f"📍 Contracts analyzed: {contracts_analyzed}")
    print(f"📅 Overall date range: {date_range}")
    print(f"🔧 Unique methods across all contracts: {unique_methods}")
    print(f"💰 Total fees paid: {total_fees:,}")
    
    print(f"\n📊 Transactions by contract:")
    contract_counts = transactions_df['contract_name'].value_counts()
    for contract, count in contract_counts.items():
        print(f"  {contract}: {count}")
    
    print(f"\n📈 Top methods across all contracts:")
    method_counts = transactions_df['method'].value_counts()
    for method, count in method_counts.head(10).items():
        print(f"  {method}: {count}")
    
    print(f"\n🔍 Sample transactions from combined dataset:")
    sample_cols = ['contract_name', 'timestamp_', 'method', 'fee_value', 'param_0_name', 'param_0_value']
    print(transactions_df[sample_cols].head(10).to_string(index=False))
    
    # Individual contract DataFrames (for separate analysis)
    store_df = transactions_df[transactions_df['contract_name'] == 'Store'].copy()
    donations_df = transactions_df[transactions_df['contract_name'] == 'Donations'].copy()
    
    print(f"\n💡 Individual DataFrames created:")
    print(f"  store_df: {len(store_df)} transactions")
    print(f"  donations_df: {len(donations_df)} transactions")
    print(f"  transactions_df: {len(transactions_df)} transactions (combined)")
    
else:
    print("❌ No transactions collected from any contract")
    transactions_df = pd.DataFrame()
    store_df = pd.DataFrame()
    donations_df = pd.DataFrame()

print(f"\n🎉 Multi-contract data collection complete!")


📡 Fetching transactions for Store
📍 Contract: 0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE
Fetching page 1 for Store...
Found 50 transactions on page 1
✅ Added 50 new transactions for Store
Fetching page 2 for Store...
Found 50 transactions on page 2
✅ Added 50 new transactions for Store
Fetching page 3 for Store...
Found 1 transactions on page 3
✅ Added 1 new transactions for Store
📄 Reached last page for Store

📊 Store Summary:
  Total pages: 3
  Total transactions: 0

⏳ Waiting before fetching next contract...

📡 Fetching transactions for Donations
📍 Contract: 0x6aD9E8e5236C0E2cF6D755Bb7BE4eABCbC03f76d
Fetching page 1 for Donations...
Found 50 transactions on page 1
✅ Added 50 new transactions for Donations
Fetching page 2 for Donations...
Found 50 transactions on page 2
✅ Added 50 new transactions for Donations
Fetching page 3 for Donations...
Found 50 transactions on page 3
✅ Added 50 new transactions for Donations
Fetching page 4 for Donations...
Found 50 transactions on page 4
✅ 

In [11]:
MARKET_MAP = {
    'Brink' : 'Brink',
    'SheFi' : 'SheFi',
    '1001' : 'ledger_nano_x',
    '1002' : 'ledger_stax', 
    '1003' : 'bitrefill_25',
    '1004' : 'bitrefill_50',
    '1005' : 'bitrefill_100', 
    '1006' : 'bitrefill_200',
    '1007' : 'bitrefill_1000'
}

In [35]:
transactions_df = format_musd_currency_columns(transactions_df, ['fee_value'])
transactions_df['timestamp_'] = pd.to_datetime(transactions_df['timestamp_'])

In [36]:
transactions_df.dtypes

contract_name                    object
contract_address                 object
timestamp_          datetime64[ns, UTC]
method                           object
fee_value                       float64
has_error                        object
from_address                     object
to_address                       object
transactionHash_                 object
block_number                      int64
param_0_name                     object
param_0_value                    object
timestamp_dt        datetime64[ns, UTC]
date                date64[ms][pyarrow]
dtype: object

In [37]:
transactions_df['date'] = transactions_df['timestamp_'].astype('date64[pyarrow]')

In [40]:
txns_with_fees = transactions_df.loc[transactions_df['fee_value'] != 0]

In [41]:
txns_with_fees['method'].unique()

array(['donateWithPermit', 'orderWithPermit', 'transferOwnership',
       'updateStoreTreasury', 'updateStoreManager',
       'updateProductPerCustomerLimit', 'updateProductStock',
       'updateProductPrice', None, 'updateBeneficiary'], dtype=object)

In [42]:
txns_with_fees.loc[txns_with_fees['method'].isin(['transferOwnership',
       'updateStoreTreasury', 'updateStoreManager',
       'updateProductPerCustomerLimit', 'updateProductStock',
       'updateProductPrice', None, 'updateBeneficiary'])]

,contract_name,contract_address,timestamp_,method,fee_value,has_error,from_address,to_address,transactionHash_,block_number,param_0_name,param_0_value,timestamp_dt
79,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:49.000000Z,transferOwnership,1424304000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0xf5b5a3781a858a0e122d9b0889fd1232ee45202c9bc6...,269576,newOwner,0x98D8899c3030741925BE630C710A98B57F397C7a,2025-05-16 11:06:49+00:00
80,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:41.000000Z,updateStoreTreasury,1407780000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x8d1e61a6e455c70a2e89ed43bd07728f6e2fcfef78ac...,269574,newStoreTreasury,0x34A7b87835bE0c66a270ed2646AB797239AfdbeD,2025-05-16 11:06:41+00:00
81,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:34.000000Z,updateStoreManager,945756000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x96b2844a66694516a10788a1fefe37e18207e65c4f60...,269572,newStoreManager,0xF4Ca53e7dc721E156cf6c15E130019c9C0d39CA3,2025-05-16 11:06:34+00:00
82,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:31.000000Z,updateProductPerCustomerLimit,881658000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x4e136bdaf6805aa46d7099de710c5ed4f0b2ff1e0b05...,269571,productId,1006,2025-05-16 11:06:31+00:00
83,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:23.000000Z,updateProductStock,957312000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x4db32f9138bdbe4033151ee7779ad4b87d112fadaabc...,269570,productId,1006,2025-05-16 11:06:23+00:00
84,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:19.000000Z,updateProductPrice,1420686000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0xb408df76ee6cb2d39ba3ea2ff23239d4f84fe6c98e60...,269569,productId,1006,2025-05-16 11:06:19+00:00
85,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:12.000000Z,updateProductPerCustomerLimit,881658000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x8bbd96001cab83477b226208b780ed462c0fe2ccb3f7...,269567,productId,1005,2025-05-16 11:06:12+00:00
86,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:06:05.000000Z,updateProductStock,957312000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x6e58ab6f623af82275419081bf4f42946430d859c8e9...,269565,productId,1005,2025-05-16 11:06:05+00:00
87,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:05:58.000000Z,updateProductPrice,1420686000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0x9501887a7e076d0b11b64bb28672f596b7de40a1a6ff...,269563,productId,1005,2025-05-16 11:05:58+00:00
88,Store,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,2025-05-16T11:05:51.000000Z,updateProductPerCustomerLimit,881658000000,None,0x123694886DBf5Ac94DDA07135349534536D14cAf,0xB6881e8b21a3cd6D23c4F90724E26e35BB8980bE,0xcc7008d42616511d722390151b1288c3f5431f5ad8c3...,269561,productId,1004,2025-05-16 11:05:51+00:00


In [33]:
final_txns_df = transactions_df.loc[transactions_df['method'].isin(['orderWithPermit', 'donateWithPermit'])]

In [34]:
final_txns_df[['date', 'contract_name', 'param_0_name', 'param_0_value', 
                 'fee_value', 'method', 'transactionHash_']]

,date,contract_name,param_0_name,param_0_value,fee_value,method,transactionHash_
0,2025-08-12,Store,productId,1003,1.536400e-07,orderWithPermit,0x467e5721410f71d4d23093a368316adbf8ff7251bcb2...
101,2025-08-11,Donations,beneficiaryId,Brink,1.498805e-07,donateWithPermit,0x501d9f570475a2aab097d27b51328304411f8f0d72ea...
1,2025-08-10,Store,productId,1005,1.786488e-07,orderWithPermit,0xf32b96380e7d7c7467e2c6a24b7b3dc29c08f743dc8f...
2,2025-08-10,Store,productId,1006,1.843680e-07,orderWithPermit,0x79fd72be2540179dd349fe43ceef3afb424d74594f55...
102,2025-08-08,Donations,beneficiaryId,Brink,1.373905e-07,donateWithPermit,0xf8b7133c90681b54765deea0dda749c42b2239eab50b...
...,...,...,...,...,...,...,...
466,2025-05-23,Donations,beneficiaryId,SheFi,2.767554e-06,donateWithPermit,0xdff265922c8348c2642e0cb8b477634bdd236e512451...
467,2025-05-23,Donations,beneficiaryId,Brink,2.095691e-06,donateWithPermit,0x14d0cddf04c1f70c115d10dc3a01ca1539182542c7ab...
77,2025-05-23,Store,productId,1003,4.135814e-06,orderWithPermit,0x03b4bf8a76f275a474751441d0d48957b2b7a3ff01d6...
78,2025-05-22,Store,productId,1003,3.759507e-06,orderWithPermit,0x1b950a8d080b6919661f96b1c76b6452dde1e3327773...
